In [9]:

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np


# Load dataset

df = pd.read_csv("")

# Ensure you have a 'label' column: 1 = accident, 0 = normal
if 'label' not in df.columns:
    raise ValueError("Dataset must contain a 'label' column (0=normal, 1=accident)")

X = df.drop(columns=['label']).values
y = df['label'].values.astype(np.float32)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)


# Define Neural Network (similar to your TFLite one but deeper)

class AccidentNet(nn.Module):
    def __init__(self, input_dim):
        super(AccidentNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # Output probability [0–1]
        )

    def forward(self, x):
        return self.net(x)

model = AccidentNet(input_dim=X_train.shape[1])


# Train
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 50

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {loss.item():.4f}")


# Evaluate

model.eval()
with torch.no_grad():
    preds = model(X_test_t)
    preds_class = (preds >= 0.5).float()
    acc = accuracy_score(y_test, preds_class)
    f1 = f1_score(y_test, preds_class)
    print("\n✅ Accuracy:", acc)
    print("✅ F1 Score:", f1)
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds_class))

# -----------------------------------------------------
# 5️⃣ Export to ONNX (for Android)
# -----------------------------------------------------
dummy_input = torch.randn(1, X_train.shape[1], dtype=torch.float32)
onnx_path = "driver_behavior_model.onnx"
torch.onnx.export(
    model, dummy_input, onnx_path,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
    opset_version=17
)
print(f"\n📦 Model exported successfully to {onnx_path}")


# Save feature scaler (optional, for consistent preprocessing)

import joblib
joblib.dump(scaler, "scaler.pkl")
print("📁 Saved feature scaler as scaler.pkl")


Epoch [10/50] Loss: 0.6483
Epoch [20/50] Loss: 0.5794
Epoch [30/50] Loss: 0.4889
Epoch [40/50] Loss: 0.3959
Epoch [50/50] Loss: 0.3124

✅ Accuracy: 0.9502262443438914
✅ F1 Score: 0.9447236180904522
Confusion Matrix:
 [[116   2]
 [  9  94]]


C:\temp\ipykernel_39900\787738547.py:99: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1104 21:04:03.359000 39900 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W1104 21:04:04.853000 39900 site-packages\torch\onnx\_internal\exporter\_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `AccidentNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `AccidentNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅

📦 Model exported successfully to driver_behavior_model.onnx
📁 Saved feature scaler as scaler.pkl


In [2]:
import pandas as pd

df = pd.read_csv("features_14.csv")

# Create dummy labels for testing (first half = normal, second half = accident)
df["label"] = [0]* (len(df)//2) + [1]* (len(df) - len(df)//2)

df.to_csv("features_14_labeled.csv", index=False)
print("✅ Added dummy label column and saved as features_14_labeled.csv")


✅ Added dummy label column and saved as features_14_labeled.csv


In [8]:
                        pip install onnx onnxruntime onnxscript


   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.5 MB 2.1 MB/s eta 0:00:08
   -- ------------------------------------- 1.0/16.5 MB 2.6 MB/s eta 0:00:06
   --- ------------------------------------ 1.6/16.5 MB 2.5 MB/s eta 0:00:06
   ----- ---------------------------------- 2.4/16.5 MB 2.7 MB/s eta 0:00:06
   ------- -------------------------------- 2.9/16.5 MB 2.6 MB/s eta 0:00:06
   -------- ------------------------------- 3.4/16.5 MB 2.7 MB/s eta 0:00:05
   ---------- ----------------------------- 4.2/16.5 MB 2.7 MB/s eta 0:00:05
   ---------- ----------------------------- 4.5/16.5 MB 2.6 MB/s eta 0:00:05
   ------------ --------------------------- 5.2/16.5 MB 2.7 MB/s eta 0:00:05
   -------------- ------------------------- 5.8/16.5 MB 2.7 MB/s eta 0:00:04
   --------------- ------------------------ 6.3/16.5 MB 2.7 MB/s eta 0:00:04
   ---------------- ----------------------- 6.8/16.5 MB 2.6 MB/s eta 0:00:04
   ---